In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from math import ceil
from copy import deepcopy
import numpy as np

In [2]:
G = nx.generators.gnp_random_graph(50, 0.3)

In [3]:
def node_cost(n: int, G: nx.Graph) -> float:
    return 1.0

## Euristiche

In [4]:
def f1(S: set[int], G: nx.Graph) -> float:
    result = 0
    for v in G.nodes():
        n_v = set(nx.neighbors(G, v))
        result += min(len(n_v.intersection(S)), ceil(len(n_v) / 2))
    return result

## Algoritmi di diffusione

In [5]:
def cost_seeds_greedy(G: nx.Graph, k: int, c: callable, f: callable) -> tuple[set[int], float]:
    S_p = set()
    S_d = set()
    while True:
        nodes = list(set(G.nodes()).difference(S_d))
        candidates = [
            (f(S_d.union({v}), G) - f(S_d, G)) / c(v, G)
            for v in nodes
        ]
        u = nodes[candidates.index(max(candidates))]
        S_p = deepcopy(S_d)
        S_d = S_p.union({u})
        if sum([c(n, G) for n in S_d]) > k:
            break
    return S_p, sum([c(n, G) for n in S_p])

In [10]:
def wtss(G: nx.Graph, k: int, c: callable, f: callable, t: np.ndarray):
    S = set()
    _G = deepcopy(G)
    d = np.array(list(dict(nx.degree(G)).values()))
    _t = t.copy()
    while len(_G.nodes()) > 0:
        activated = [i for i in (t == 0).nonzero()[0].tolist() if i in _G.nodes()]
        candidates = [i for i in (d < t).nonzero()[0].tolist() if i in _G.nodes()]
        if len(activated) > 0:
            v = activated[0]
            for u in nx.neighbors(_G, v):
                _t[u] = max(_t[u] - 1, 0)
        elif len(candidates) > 0:
            v = candidates[0]
            # Check the costs
            if sum([c(n, _G) for n in S]) + c(v, _G) > k:
                break
            S.add(v)
            for u in nx.neighbors(_G, v):
                _t[u] = _t[u] - 1
        else:
            candidates = {
                u: (c(u, _G) * t[u])/(d[u] * (d[u] + 1))
                for u in _G.nodes()
            }
            v = max(candidates, key=candidates.get)
        for u in nx.neighbors(_G, v):
            d[u] = d[u] - 1
        _G.remove_node(v)
    return S, sum([c(n, G) for n in S])

In [11]:
t = np.ceil(np.array(list(dict(nx.degree(G)).values())) / 2)
k = 10
wtss(G, k, node_cost, f1, t)

({0, 5, 6, 7, 9, 17, 18, 20, 22, 42}, 10.0)